<a href="https://colab.research.google.com/github/mbeck33/dsportfolio/blob/master/SOTUS2HardTryNewsGroups_ClassExample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sample NLP Code for IST 718



In [ ]:
# A quick sample for using Universal Language Model Fine-Tuning for Text Classification
# aka ULMFiT
# Check out nlp.fast.ai for more

# import libraries
import fastai
from fastai import *
from fastai.text import * 
import pandas as pd
import numpy as np
from functools import partial
import io
import os

In [ ]:
!pip install torch_nightly -f https://download.pytorch.org/whl/nightly/cu92/torch_nightly.html
!pip install fastai

In [ ]:
# OBTAIN

from sklearn.datasets import fetch_20newsgroups
dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers', 'footers', 'quotes'))
documents = dataset.data

In [ ]:
# OBTAIN

df = pd.DataFrame({'label':dataset.target, 'text':dataset.data})

In [ ]:
df.shape

(11314, 2)

Let's try looking at just Politics and Windows articles - source?  https://kdd.ics.uci.edu/databases/20newsgroups/20newsgroups.data.html


In [ ]:
# SCRUB

df = df[df['label'].isin([5,18])]
df = df.reset_index(drop = True)

In [ ]:
df['label'].value_counts()


5     593
18    465
Name: label, dtype: int64

In [ ]:
# SCRUB - TEXT

df['text'] = df['text'].str.replace("[^a-zA-Z]", " ")


In [ ]:
# SCRUB - STOPWORDS
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords 
stop_words = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# SCRUB - TOKEN

# tokenization 
tokenized_doc = df['text'].apply(lambda x: x.split())

# remove stop-words 
tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words])

# de-tokenization 
detokenized_doc = [] 
for i in range(len(df)): 
    t = ' '.join(tokenized_doc[i]) 
    detokenized_doc.append(t) 

df['text'] = detokenized_doc

In [ ]:
# SCRUB - TEST & TRAIN

from sklearn.model_selection import train_test_split

# split data into training and validation set
df_trn, df_val = train_test_split(df, stratify = df['label'], test_size = 0.4, random_state = 12)

In [ ]:
# SCRUB - CHECK SPLIT

df_trn.shape, df_val.shape

((634, 2), (424, 2))

In [ ]:
# MODEL

# Language model data
data_lm = TextLMDataBunch.from_df(train_df = df_trn, valid_df = df_val, path = "")

# Classifier model data
data_clas = TextClasDataBunch.from_df(path = "", train_df = df_trn, valid_df = df_val, vocab=data_lm.train_ds.vocab, bs=32)

In [ ]:
# MODEL - TUNE PRE-TRAINED MODEL

learn = language_model_learner(data_lm, AWD_LSTM,pretrained=True, drop_mult=0.7)

In [ ]:
# MODEL - LEARNING RATE

# train the learner object with learning rate = 1e-2
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,6.853075,5.499538,0.200233,00:05


In [ ]:
# MODEL - SAVE

learn.save_encoder('ft_enc')

In [ ]:
# MODEL - BUILD CLASSIFIER

learn = text_classifier_learner(data_clas, AWD_LSTM,drop_mult=0.7)
learn.load_encoder('ft_enc')

RNNLearner(data=TextClasDataBunch;

Train: LabelList (634 items)
x: TextList
xxbos i recently compiled x r pl sources using gcc xxmaj sun xxmaj everything seems work fine xxmaj usually xxmaj but seemingly random times server hang i click mouse somewhere never happens back turned without warning freeze requiring server killed xxmaj sometimes run fine weeks sometimes xxunk xxmaj os frame buffer xxunk xxmaj has anyone seen ideas anything thanks jeff,xxbos xxmaj you right i xxunk xxmaj xxunk xxmaj xxunk xxmaj nevertheless i would suggest xxup you difference proposed xxup bill xxunk xxmaj congress executive order crammed xxup our xxup throats xxmaj do disagree xxmaj went xxmaj post xxmaj office xxmaj friday got passport apps,xxbos i two xxmaj motif xxmaj widgets i would like control one via keyboard mouse i set keyboard focus first widget soon i click mouse second one i lose keyboard focus first one xxmaj could kind soul show xxmaj thanks,xxbos a simple question xxmaj xperts xxmaj is possib

In [ ]:
# MODEL - FIT

learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.335721,0.253580,0.889151,00:08


In [ ]:
# MODEL - EVALUATE

preds, targets = learn.get_preds()

predictions = np.argmax(preds, axis = 1)
pd.crosstab(predictions, targets)

col_0,0,1
row_0,,
0,234,43
1,4,143


tensor([1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1,
        1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1,
        1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1,
        0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0,
        0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0,
        0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0,
        0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1,
        0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1,
        0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1,
        1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0,
        0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0,
        0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0,